#This script is to run raygun to see what esm2 mask replacement and raygun fixed length embeddings to form final model do when partial sequences are run.

Write functions to run step by step parts of the raygun model and save the intermediates

In [1]:
import torch
print(torch.cuda.is_available())   # Should be True if CUDA works
print(torch.cuda.device_count())   # Number of GPUs
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
1
NVIDIA RTX A6000


In [3]:
# esm-2 model
from esm.pretrained import esm2_t33_650M_UR50D
esmmodel, alph = esm2_t33_650M_UR50D()
bc             = alph.get_batch_converter()
esmmodel       = esmmodel.to(0)
esmmodel.eval()

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [4]:
# data is a list of (id,sequence) pairs
data = [("egfp", "MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITLGMDELYK")]
_, _, tok = bc(data)
# return esmemb
esmemb       = esmmodel(tok.to(0), repr_layers = [33],
                        return_contacts=False)["representations"][33][:,1:-1]  #remember to remove the start and end tokens!
# esmemb.shape should be torch.Size([1, 239, 1280])

In [5]:
print(esmemb.shape)
#length of the sequence
print(len(data[0][1]))
import numpy as np
import matplotlib.pyplot as plt 
#plt.imshow(esmemb.cpu().detach().numpy()[0])
print(tok.shape)
#plt.imshow(tok.detach().numpy())

torch.Size([1, 239, 1280])
239
torch.Size([1, 241])


In [8]:
from raygun.pretrained import raygun_4_4mil_800M

# Load the bare PyTorch model (not Lightning)
raymodel = raygun_4_4mil_800M(return_lightning_module=False)

# Print the model architecture
raymodel.eval()


Using cache found in /hpc/home/khs36/.cache/torch/hub/rohitsinghlab_raygun_main


Raygun(
  (encoder): RaygunEncoder(
    (encoders): ModuleList(
      (0-11): 12 x Block(
        (encoder): TransformerLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (rot_emb): RotaryEmbedding()
          )
          (self_attn_layer_norm): ESM1LayerNorm()
          (fc1): Linear(in_features=1280, out_features=2560, bias=True)
          (fc2): Linear(in_features=2560, out_features=1280, bias=True)
          (final_layer_norm): ESM1LayerNorm()
        )
        (convblock): ConvBlock(
          (c1): ConvMasked(
            (conv): Conv1d(1280, 640, kernel_size=(7,), stride=(1,), padding=valid)
          )
          (s1): SiLU()
          (c2): ConvMasked(
          

#Here to put the model specification code: 



Raygun(
  (encoder): RaygunEncoder(
    (encoders): ModuleList(
      (0-11): 12 x Block(
        (encoder): TransformerLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (rot_emb): RotaryEmbedding()
          )
          (self_attn_layer_norm): ESM1LayerNorm()
          (fc1): Linear(in_features=1280, out_features=2560, bias=True)
          (fc2): Linear(in_features=2560, out_features=1280, bias=True)
          (final_layer_norm): ESM1LayerNorm()
        )
        (convblock): ConvBlock(
          (c1): ConvMasked(
            (conv): Conv1d(1280, 640, kernel_size=(7,), stride=(1,), padding=valid)
          )
          (s1): SiLU()
          (c2): ConvMasked(
          